# Añadiendo la posibilidad de usar multiples veces el mismo tensor
Como se vio al final del [notebook](02-intro-autograd.ipynb) anterior, si un tensor participa en la creacion de más de un tensor, su gradiente no se acumula, simplemente sobreescribe el gradiente con el ultimo gradiente recibido por el tensor. 

Para que un tensor pueda participar en la creacion de más de un tensor y mantener correctamente su gradiente es necesario añadir una nueva funcion y actualizar otras tres.

Primero que nada los gradientes tienen que poder ser acumulables, permitiendo que si un tensor es usado más de una vez, pueda recibir el gradiente de todos sus hijos (tensores que se originan a partir de el)

In [115]:
import numpy as np

class Tensor(object):
    
    def __init__(self, data, 
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        '''
        Inicializa un tensor utilizando numpy
        
        @data: una lista de numeros
        @creators: lista de tensores que participarion en la creacion de un nuevo tensor
        @creators_op: la operacion utilizada para combinar los tensores en el nuevo tensor
        '''
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
        self.autograd = autograd
        self.children = {}
        # se asigna un id al tensor
        if(id is None):
            id = np.random.randint(0,100000)
        self.id = id
        
        # se hace un seguimiento de cuantos hijos tiene un tensor
        # si los creadores no es none
        if (creators is not None):
            # para cada tensor padre
            for c in creators:
                # se verifica si el tensor padre posee el id del tensor hijo
                # en caso de no estar, agrega el id del tensor hijo al tensor padre
                if(self.id not in c.children):
                    c.children[self.id] = 1
                # si el tensor ya se encuentra entre los hijos del padre
                # y vuelve a aparece, se incrementa en uno
                # la cantidad de apariciones del tensor hijo
                else:
                    c.children[self.id] += 1
                    
    def all_children_grads_accounted_for(self):
        '''
        Verifica si un tensor ha recibido la cantidad 
        correcta de gradientes por cada uno de sus hijos
        '''
        # print('tensor id:', self.id)
        for id, cnt in self.children.items():
            print(' all_children_grads_accounted_for id:',id,'cnt:', cnt)
            if(cnt != 0):
                return False
        return True
        
    def backward(self, grad, grad_origin=None):
        '''
        Funcion que propaga recursivamente el gradiente a los creators o padres del tensor
        
        @grad: gradiente 
        @grad_orign
        '''
        if(self.autograd):
            if(grad_origin is not None):
                # Verifica para asegurar si se puede hacer retropropagacion
                if(self.children[grad_origin.id] == 0):
                    raise Exception("No se puede retropropagar mas de una vez")
                # o si se está esperando un gradiente, en dicho caso se decrementa
                else:
                    # el contador para ese hijo
                    self.children[grad_origin.id] -= 1
        
        # acumula el gradiente de multiples hijos
        if(self.grad is None):
            self.grad = grad
        else:
            self.grad += grad
        
        
        print('\nTensor id',self.id, '\n',
              'Has creators?', self.creators is not None, '\n',
              'All children grads accounted for is', self.all_children_grads_accounted_for(), '\n',
              'grad origin is None?', grad_origin is None
             )
        if(self.creators is not None and
          (self.all_children_grads_accounted_for() or grad_origin is None)):
            
            if (self.creation_op == 'add'):
                # al recibir self.grad, empieza a realizar backprop
                print('', self.id, 'creators are:')
                print(' creator', self.creators[0].id, ':', self.creators[0], 
                      'creator', self.creators[1].id, ':',self.creators[1])
                self.creators[0].backward(self.grad, grad_origin=self)
                self.creators[1].backward(self.grad, grad_origin=self)
                
        
    def __add__(self, other):
        '''
        @other: un Tensor
        '''
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self, other],
                          creation_op='add')
        return Tensor(self.data + other.data)
    
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

In [116]:
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)
d = a + b
e = b + c
f = d + e
f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data == np.array([2,2,2,2,2]))


Tensor id 28071 
 Has creators? True 
 All children grads accounted for is True 
 grad origin is None? True
 28071 creators are:
 creator 72370 : [3 4 5 6 7] creator 71868 : [7 6 5 4 3]
 all_children_grads_accounted_for id: 28071 cnt: 0

Tensor id 72370 
 Has creators? True 
 All children grads accounted for is True 
 grad origin is None? False
 all_children_grads_accounted_for id: 28071 cnt: 0
 72370 creators are:
 creator 1 : [1 2 3 4 5] creator 42364 : [2 2 2 2 2]
 all_children_grads_accounted_for id: 72370 cnt: 0

Tensor id 1 
 Has creators? False 
 All children grads accounted for is True 
 grad origin is None? False
 all_children_grads_accounted_for id: 72370 cnt: 0
 all_children_grads_accounted_for id: 71868 cnt: 1

Tensor id 42364 
 Has creators? False 
 All children grads accounted for is False 
 grad origin is None? False
 all_children_grads_accounted_for id: 28071 cnt: 0

Tensor id 71868 
 Has creators? True 
 All children grads accounted for is True 
 grad origin is None? 

In [121]:
x = Tensor([2,2,2,2], autograd=True)
y = x + x
z = y + y
print('x',x.id)
print('y', y.id)
print('z', z.id)
z.backward(Tensor([1,1,1,1]))
print('\nx gradient data:',x.grad.data)
z.backward(Tensor([1,1,1,1]))

x 39389
y 55252
z 49032

Tensor id 49032 
 Has creators? True 
 All children grads accounted for is True 
 grad origin is None? True
 49032 creators are:
 creator 55252 : [4 4 4 4] creator 55252 : [4 4 4 4]
 all_children_grads_accounted_for id: 49032 cnt: 1

Tensor id 55252 
 Has creators? True 
 All children grads accounted for is False 
 grad origin is None? False
 all_children_grads_accounted_for id: 49032 cnt: 1
 all_children_grads_accounted_for id: 49032 cnt: 0

Tensor id 55252 
 Has creators? True 
 All children grads accounted for is True 
 grad origin is None? False
 all_children_grads_accounted_for id: 49032 cnt: 0
 55252 creators are:
 creator 39389 : [2 2 2 2] creator 39389 : [2 2 2 2]
 all_children_grads_accounted_for id: 55252 cnt: 1

Tensor id 39389 
 Has creators? False 
 All children grads accounted for is False 
 grad origin is None? False
 all_children_grads_accounted_for id: 55252 cnt: 0

Tensor id 39389 
 Has creators? False 
 All children grads accounted for is Tru

Exception: No se puede retropropagar mas de una vez